In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from torchvision.models import vgg16_bn
import PIL
import imageio
from superres import *
from scipy.ndimage.interpolation import zoom as npzoom
from skimage.util import img_as_float32, img_as_ubyte
from skimage.measure import compare_ssim, compare_psnr

In [2]:
torch.cuda.set_device(1)

In [3]:
img_data = Path('/scratch/bpho/datasets/emsynth_003/')
model_path = Path('/scratch/bpho/models')

In [4]:
def get_src():
    hr_tifs = img_data/f'hr'
    lr_tifs = img_data/f'lr_up'

    def map_to_hr(x):
        hr_name = x.relative_to(lr_tifs)
        return hr_tifs/hr_name
    print(lr_tifs)
    src = (ImageImageList
            .from_folder(lr_tifs)
            .split_by_rand_pct()
            .label_from_func(map_to_hr))
    return src


def get_data(bs, size, noise=None, max_zoom=1.1):
    src = get_src()
    tfms = get_transforms(flip_vert=True, max_zoom=max_zoom)
    data = (src
            .transform(tfms, size=size)
            .transform_y(tfms, size=size)
            .databunch(bs=bs).normalize(imagenet_stats, do_y=True))
    data.c = 3
    return data


In [5]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)
blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]

In [6]:
base_loss = F.l1_loss

class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [7]:
feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])

In [7]:
bs = 64
size = 128
data = get_data(bs, size, max_zoom=6)
data

/scratch/bpho/datasets/emsynth_003/lr_up


ImageDataBunch;

Train: LabelList (79998 items)
x: ImageImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
Path: /scratch/bpho/datasets/emsynth_003/lr_up;

Valid: LabelList (19999 items)
x: ImageImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
Path: /scratch/bpho/datasets/emsynth_003/lr_up;

Test: None

In [ ]:
data.show_batch(3, ds_type=DatasetType.Valid)

In [8]:
arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

0

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [8]:
lr = 1e-3

In [9]:
def do_fit(save_name, lrs=slice(lr), pct_start=0.9, cycle_len=10):
    learn.fit_one_cycle(cycle_len, lrs, pct_start=pct_start)
    learn.save(save_name)
    num_rows = min(learn.data.batch_size, 3)
    learn.show_results(rows=num_rows, imgsize=5)

In [ ]:
do_fit('emsynth_003_unet.0', lr, cycle_len=3)

epoch,train_loss,valid_loss,mse_loss,ssim,psnr,pixel,feat_0,feat_1,feat_2,gram_0,gram_1,gram_2,time
0,1.514454,1.370362,0.037342,0.672077,14.279596,0.151167,0.186773,0.213862,0.052719,0.249625,0.470949,0.045266,46:39


In [ ]:
learn.unfreeze()

In [ ]:
do_fit('emsynth_003_unet.1', slice(1e-5,lr), cycle_len=3)

In [ ]:
bs = 32
size = 256
data = get_data(bs, size)

arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

learn = learn.load('emsynth_003_unet.1')

In [ ]:
do_fit('emsynth_003_unet.2', lr, cycle_len=3)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit('emsynth_003_unet.3', slice(1e-5,lr), cycle_len=3)

In [ ]:
bs = 4
size = 512
data = get_data(bs, size)

arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

learn = learn.load('emsynth_003_unet.3')

In [ ]:
do_fit('emsynth_003_unet.4', lr)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit('emsynth_003_unet.5', slice(1e-5,lr), cycle_len=10)

In [ ]:
bs = 2
size = 1024
data = get_data(bs, size)

arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

learn = learn.load('emsynth_003_unet5')

In [ ]:
do_fit('emsynth_003_unet.6', lr/100000, cycle_len=4)

In [ ]:
learn.unfreeze()

In [ ]:
do_fit('emsynth_003_unet.7', slice(1e-5,lr), cycle_len=10)

In [ ]:
bs = 1
size = 1024
data = get_data(bs, size)

arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

learn = learn.load('emsynth_003_unet.6').to_fp16()

In [ ]:
learn.unfreeze()
do_fit('emsynth_003_unet.8', slice(1e-6,1e-5), cycle_len=10)

In [ ]:
do_fit('emsynth_003_unet.9', slice(1e-6,1e-5), cycle_len=1)

In [10]:
!ls /scratch/bpho/models

combo002_unet.0.pth	     nuloss.3.pth
combo002_unet.1.pth	     nuloss.4.pth
combo002_unet.2.pth	     nuloss.5.pth
combo002_unet.3.pth	     nuloss.6.pth
combo002_unet.4.pth	     nuloss.7.pth
combo002_unet.5.pth	     nuloss_nopt.1.pth
combo002_unet.6.pth	     nuloss_nopt.2.pth
combo002_unet.7.pth	     paired_001_unet.0.pth
emsynth_002_unet.0.pth	     paired_001_unet.1.pth
emsynth_002_unet.1.pth	     paired_001_unet.2.pth
emsynth_002_unet.2.pth	     paired_001_unet.3.pth
emsynth_002_unet.3.pth	     paired_001_unet.4.pth
emsynth_002_unet.4.pth	     paired_001_unet.5.pth
emsynth_002_unet.5.pth	     paired_001_unet.6.pth
emsynth_002_unet.8.pth	     paired_001_unet.7.pth
emsynth_002_unet.9.pth	     paired_001_unet.8.pkl
emsynth_003_unet.0.pth	     paired_001_unet.8.pth
emsynth_003_unet.1.pth	     paired_001_unet_lr2hr.0.pth
emsynth_003_unet.2.pth	     paired_001_unet_lr2hr.1.pth
emsynth_003_unet.3.pth	     paired_001_unet_lr2hr.2.pth
emsynth_003_unet.4.pth	     paired_001_unet_lr2hr.3.pkl
mit

In [11]:
bs = 1
size = 1920
data = get_data(bs, size)

arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, metrics=superres_metrics, 
                     callback_fns=LossMetrics, blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

learn = learn.load('emsynth_003_unet.4')

/scratch/bpho/datasets/emsynth_003/lr_up


/home/fredmonroe/anaconda3/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type FeatureLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [12]:
test_files = Path('/scratch/bpho/datasources/EM_manually_aquired_pairs_01242019/')
test_hr = list((test_files/'aligned_hr').glob('*.tif'))
test_lr = list((test_files/'aligned_lr').glob('*.tif'))
results = Path('/scratch/bpho/results/emsynth_crap')

if results.exists(): shutil.rmtree(results)
results.mkdir(parents=True, mode=0o775, exist_ok=True)

def get_key(fn):
    return fn.stem[0:(fn.stem.find('Region')-1)]

hr_map = { get_key(fn): fn for fn in test_hr }
lr_map = { get_key(fn): fn for fn in test_lr }


In [13]:
ssims = []
psnrs = []
for k in progress_bar(hr_map):
    hr_fn, lr_fn = hr_map[k], lr_map[k]
    hr_img = PIL.Image.open(hr_fn)
    lr_img = PIL.Image.open(lr_fn)
    lr_img_data = img_as_float32(lr_img)
    lr_up_data = npzoom(lr_img_data, 4, order=1)
    lr_up_img = Image(tensor(lr_up_data[None]))
    hr_pred_img, aaa, bbb = learn.predict(lr_up_img)
    pred_img = PIL.Image.fromarray(img_as_ubyte(np.array(hr_pred_img.data))[0,:,:])
    
    lr_img.save(results/f'{k}_orig.tif')
    hr_img.save(results/f'{k}_truth.tif')
    pred_img.save(results/f'{k}_pred.tif')
    hr_img_data = np.array(hr_img)
    
    ssims.append(compare_ssim(img_as_float32(np.array(hr_img)), img_as_float32(np.array(pred_img))))
    psnrs.append(compare_psnr(img_as_float32(np.array(hr_img)), img_as_float32(np.array(pred_img))))

/home/fredmonroe/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [14]:
np.array(ssims).mean(), np.array(psnrs).mean()

(0.7273812282498017, 20.06313527537938)

In [15]:
import skimage.io as io

#target_path = Path('/DATA/Dropbox/bpho_movie_results/emsynth_003/')
target_path = results

orig,tru,pred = [list(target_path.glob(f'*{tag}*')) for tag in ['orig','tru','pred']]
orig.sort()
tru.sort()
pred.sort()


ssims = []
c_ssims = []
l_ssims = []
psnrs = []
c_psnrs = []
l_psnrs = []

for o, t,p in progress_bar(list(zip(orig, tru,pred))):
    oimg, timg, pimg = [img_as_float32(io.imread(fn)) for fn in [o,t,p]]
    if len(pimg.shape) == 3: pimg = pimg[:,:,0]
    cimg = npzoom(oimg, 4)
    limg = npzoom(oimg, 4, order=1)
    
    ssims.append(compare_ssim(timg, pimg))
    c_ssims.append(compare_ssim(timg, cimg))
    l_ssims.append(compare_ssim(timg, limg))
    psnrs.append(compare_psnr(timg, pimg))
    c_psnrs.append(compare_psnr(timg, cimg))
    l_psnrs.append(compare_psnr(timg, limg))
np.array(ssims).mean(), np.array(psnrs).mean(), "vs", np.array(l_ssims).mean(),

(0.7273812282498016, 20.063135275379384, 'vs', 0.7308102850426267)

In [16]:
import pandas as pd

df = pd.DataFrame(dict(ssim=ssims, psnr=psnrs, 
                        bicubic_ssim=c_ssims, bicubic_psnr=c_psnrs,
                        bilinear_ssim=l_ssims, bilinear_psnr=l_psnrs))

In [17]:
df.describe()

,ssim,psnr,bicubic_ssim,bicubic_psnr,bilinear_ssim,bilinear_psnr
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000
mean,0.727381,20.063135,0.692300,19.892473,0.730810,20.155426
std,0.030599,3.424989,0.036899,3.397313,0.036152,3.544903
min,0.660067,13.204294,0.620273,14.207584,0.658729,14.275473
25%,0.704023,16.723397,0.662589,16.520311,0.702204,16.623836
50%,0.736428,19.753729,0.705555,19.531696,0.744095,19.738787
75%,0.753708,23.332377,0.724061,23.139879,0.761960,23.557128
max,0.774032,24.473792,0.746718,24.248150,0.785502,24.759367
